<a href="https://colab.research.google.com/github/apspatz/TheMonkeyCage/blob/master/The_Monkey_Cage_v03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Given a sequence of characters from this data (e.g. "Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly. 

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


This code borrows strongly from TensorFlow's tutorial on text generation using an RNN. For more information please visit TensorFlow's website directly:

https://www.tensorflow.org/tutorials/text/text_generation


Setup:

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

TensorFlow 2.x selected.


Paths to the Homer text file (which contains both the Odyssey & the Illiad):

In [2]:
path_to_file_homer = tf.keras.utils.get_file('HomerComplete_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/HomerComplete_edited.txt')

1458176/1456151 [==============================] - 1s 0us/step


Path to the Yoda text file

In [3]:
path_to_file_yoda = tf.keras.utils.get_file('yoda_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/yoda_edited.txt')

8192/6498 [=====================================] - 0s 0us/step


Path to the Shakespeare text file:

In [3]:
path_to_file_shakespeare = tf.keras.utils.get_file('shakespeare_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/shakespeare_edited.txt')

5464064/5458199 [==============================] - 1s 0us/step


Path to the Bible text file:

In [4]:
path_to_file_bible = tf.keras.utils.get_file('bible_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/bible_edited.txt')

4341760/4337219 [==============================] - 1s 0us/step


Path to the Sherlock text file:

In [5]:
path_to_file_sherlock = tf.keras.utils.get_file('sherlock_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/sherlock_edited.txt')

581632/577376 [==============================] - 0s 1us/step


Path to the Dante text files (containing The Divine Comedy, Paradise, The Divine Comedy, Purgatory, and The Vision of Hell):

In [6]:
path_to_file_dante = tf.keras.utils.get_file('DivineComedyComplete_edited.txt', 'https://mandrewsbucket.s3.amazonaws.com/DivineComedyComplete_edited.txt')

622592/622204 [==============================] - 0s 1us/step


In [5]:
# Read, then decode for py2 compat.
# text = open(path_to_file_homer, 'rb').read().decode(encoding='utf-8')
text = open(path_to_file_yoda, 'rb').read().decode(encoding='utf-8')
# text = open(path_to_file_bible, 'rb').read().decode(encoding='utf-8')
# text = open(path_to_file_dante, 'rb').read().decode(encoding='utf-8')
# text = open(path_to_file_sherlock, 'rb').read().decode(encoding='utf-8')
# text = open(path_to_file_shakespeare, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 6498 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

The very Republic is threatened, if involved the Sith are.
Hard to see, the dark side is. Discover who this assassin is, we must.
With this Naboo queen you must stay, Qui-Gon. Protect her.
May the Force be with you.
(Contd) Master Qui-Gon more to


In [7]:
print(text[-550:])

ven your physical self, perhaps.
Hmm. That face you make. Look I so old to young eyes?
Soon will I rest. Yes, forever sleep. Earned it, I have.
Strong am I with the Force... but not that strong! Twilight is upon me 
No more training do you require. Already know you that which you need.
Mmm... rest I need. Yes... rest.
Your father he is.
Told you, did he?
Unexpected this is, and unfortunate...
Remember, a Jedis strength flows from the Force.  But beware.  Anger, 
Luke...Luke...Do not...Do not underestimate the powers of the Emperor, 



In [0]:
# text = text[251:-555]
# print(text[0:250])
# print(text[-250:])

In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

60 unique characters


### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [39]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

T
h
e
 
v


In [40]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'The very Republic is threatened, if involved the Sith are.\r\nHard to see, the dark side is. Discover w'
'ho this assassin is, we must.\r\nWith this Naboo queen you must stay, Qui-Gon. Protect her.\r\nMay the Fo'
'rce be with you.\r\n(Contd) Master Qui-Gon more to say have you?\r\nA vergence, you say?\r\nBut you do! Rre'
'vealed your opinion is.\r\nTrained as a Jedi, you request for him?\r\nTested he will be.\r\nGood, good, you'
'ng one. How feel you?\r\nAfraid are you?\r\nSee through you, we can.\r\nAfraid to lose her..I think.\r\nEveyt'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [42]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'The very Republic is threatened, if involved the Sith are.\r\nHard to see, the dark side is. Discover '
Target data: 'he very Republic is threatened, if involved the Sith are.\r\nHard to see, the dark side is. Discover w'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "B" and trys to predict the index for "O" as the next character (in the case of using Homer text file). At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [43]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 30 ('T')
  expected output: 42 ('h')
Step    1
  input: 42 ('h')
  expected output: 39 ('e')
Step    2
  input: 39 ('e')
  expected output: 2 (' ')
Step    3
  input: 2 (' ')
  expected output: 56 ('v')
Step    4
  input: 56 ('v')
  expected output: 39 ('e')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [44]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [48]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 60) # (batch_size, sequence_length, vocab_size)


In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           15360     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 60)            61500     
Total params: 4,015,164
Trainable params: 4,015,164
Non-trainable params: 0
_________________________________________________________________


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [50]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 60)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.095041


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the Training

In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=200

In [56]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 1 steps
Epoch 1/200
1/1 [==============================] - 0s 193ms/step - loss: 0.0279
Epoch 2/200
1/1 [==============================] - 1s 1s/step - loss: 0.0278
Epoch 3/200
1/1 [==============================] - 1s 1s/step - loss: 0.0253
Epoch 4/200
1/1 [==============================] - 1s 1s/step - loss: 0.0266
Epoch 5/200
1/1 [==============================] - 0s 231ms/step - loss: 0.0274
Epoch 6/200
1/1 [==============================] - 0s 266ms/step - loss: 0.0247
Epoch 7/200
1/1 [==============================] - 0s 243ms/step - loss: 0.0256
Epoch 8/200
1/1 [==============================] - 0s 407ms/step - loss: 0.0282
Epoch 9/200
1/1 [==============================] - 1s 1s/step - loss: 0.0279
Epoch 10/200
1/1 [==============================] - 0s 201ms/step - loss: 0.0252
Epoch 11/200
1/1 [==============================] - 1s 1s/step - loss: 0.0272
Epoch 12/200
1/1 [==============================] - 1s 1s/step - loss: 0.0279
Epoch 13/200
1/1 [===================

## Generate Text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [57]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_200'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [59]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            15360     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 60)             61500     
Total params: 4,015,164
Trainable params: 4,015,164
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [61]:
print(generate_text(model, start_string=u"The force"))

The forced side.
 Death is a natural part of life. Rejoice for those around you who transform into the Force. a Jedis strength flows from the Force.  But beware.  Anger, 
Luke...Luke...Do not...Do not underestight our way.
 If a special session of Congress there is, easier for us to enter the Jedi Temple it scyou, the ceald fraining I have for you.
 An old friend has learned the path to immortality.
 One who has returned forever sleep. Earned it, I have.
Strong am I with the Force... but not that strong! Twilight is upout, stysrave te be dear ofertroined, I have to say about it, Lord Sidious.
 (continuing) At an end your rule is and not short e Force, he will training I have to you?
 Careful you must be when sensing the future, Anakin. The fear of loss is a path to the dark side... fear leads to anger... anger leads to hate.. hate leads to hat ou be with toremmm... rest.
Your father he is.
Told you, did he?
Unexpected this is, and unfortunate...
Remember, is. Much anger there is in 


In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [37]:
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
